<a href="https://colab.research.google.com/github/ProjektVek/TCC-UNIP/blob/main/TCC_Standalone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1>Aprendizado de máquina aplicado à detecção de fraudes de cartão de crédito:</h1></center>

---

 <center><h2>TCC 2023 - Ciência da Computação - UNIP Campinas</h2></center>

---
## Integrantes:
  <ul>
     <li><h2>Davi Ferreira Ribeiro - F21AFG6</h2></li>
     <li><h2>Ryan Vitor Rodrigues Teixeira - N662BC4</h2></li>
     <li><h2>Victor Hugo Souza Ferreira - N587FF6</h2></li>
  </ul>
</h2>

## Standalone para testar validar transações e testar tempo de execução

### Importando bibliotecas

In [71]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from os import listdir
from random import randint
import pandas as pnd
import joblib
import time

### Lendo Dataset

In [3]:
folder = "drive/MyDrive/TCC/"
archive = "creditcard.csv"
csv_dataset = pnd.read_csv(folder+archive)

In [4]:
csv_dataset.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


### Separando Dados

In [5]:
frauds = csv_dataset[csv_dataset.Class==1]
valid_transactions = csv_dataset[csv_dataset.Class==0]

In [6]:
print(len(frauds))
print(len(valid_transactions))

492
284315


### Funções do programa:

#### Ler Transações

In [7]:
def getTransactions():
  folder = "drive/MyDrive/TCC/"
  archive = "creditcard.csv"
  return pnd.read_csv(folder+archive)

#### Escolher Modelo

In [8]:
def chooseModelMenu(models):
  print("\nEscolha uma opção de modelo:\n")
  print("0 - Cancelar")
  for i in range(0,len(models)):
    match models[i]:
      case "gaussian_nb_500.pkl":
        model = "Gaussian Naive Bayes 500 transações válidas"
      case "gaussian_nb_1000.pkl":
        model = "Gaussian Naive Bayes 1000 transações válidas"
      case "gaussian_nb_3000.pkl":
        model = "Gaussian Naive Bayes 3000 transações válidas"
      case "K_Nearest_Neighbors_500.pkl":
        model = "K-Nearest Neighbors 500 transações válidas"
      case "K_Nearest_Neighbors_1000.pkl":
        model = "K-Nearest Neighbors 1000 transações válidas"
      case "K_Nearest_Neighbors_3000.pkl":
        model = "K-Nearest Neighbors 3000 transações válidas"
      case "svc_500.pkl":
        model = "Support Vector Classifier 500 transações válidas"
      case "svc_1000.pkl":
        model = "Support Vector Classifier 1000 transações válidas"
      case "svc_3000.pkl":
        model = "Support Vector Classifier 3000 transações válidas"
      case _:
        model = models[i]
    print(f'{(i+1)} - {model}')

  return int(input("\nDigite o número do modelo desejado: "))

In [9]:
def chooseModel():
  model_folder = "drive/MyDrive/TCC/modelos/"
  models = listdir(model_folder)

  choice = -1
  while choice != 0:
    choice = chooseModelMenu(models)
    if (choice < 0) or (choice > len(models)):
      print("\nOpção Inválida, digite novamente")
    else:
      if choice > 0:
        return model_folder + models[choice-1]
      else:
        break

#### Registro Único

In [10]:
def singleRegisterInfo(dataframe, register):
  register_index = register.index.values.astype(int)[0]
  register_class = register.Class.values.astype(int)[0]
  print(f"\nÍndice do registro: {register_index + 1}")
  register_type = "Fraude"
  if register_class == 0:
    register_type = "Normal"
  print(f'Tipo do registro: {register_type}')

  if register_class == 0:
    index_list = dataframe.index[dataframe.Class==0].tolist()
    position = index_list.index(register_index)
    print(f'Posição entre os registros válidos: {position+1}')
  else:
    index_list = dataframe.index[dataframe.Class==1].tolist()
    position = index_list.index(register_index)
    print(f'Posição entre os registros fraudulentos: {position+1}')

In [33]:
def predictSingleRegister(model_location, register):
  model = joblib.load(model_location)
  start_time = time.time()
  model_prediction = model.predict(register.drop(columns=['Time', 'V5', 'V6', 'V7','V8','V13','V15', 'V20','V21','V22','V23', 'V24','V25','V26','V27','V28','Amount','Class']))
  execution_time = time.time() - start_time

  prediction = "VÁLIDA"
  if model_prediction[0] == 1:
    prediction = "FRAUDULENTA"

  prediction_accuracy = "CORRETA"
  if model_prediction[0] != register.Class.values.astype(int)[0]:
    prediction_accuracy = "ERRADA"

  print(f'O modelo previu que a transação é {prediction} e sua previsão está {prediction_accuracy}')
  print(f'Tempo de execução: {"{:.2f}".format(execution_time*1000)} Milisegundos')

In [12]:
def processSingleRegister(register):
  model_location = chooseModel()
  if model_location == None:
    return
  singleRegisterInfo(all_transactions, register)
  predictSingleRegister(model_location, register)

#### Menu Principal

In [13]:
def printMenu():
  print("\nQual operação que deseja realizar?: \n")
  print("1 - Validar transação única")
  print("2 - Validar várias transações")
  print("3 - Sair\n")
  return str(input("Digite o número da operação que deseja realizar: "))

#### Registro Aleatório

In [14]:
def randomRegisterMenu():
  print("\nEscolha uma opcão: \n")
  print("1 - Registro aleatório (fraude ou normal)")
  print("2 - Registro de fraude")
  print("3 - Registro normal")
  print("4 - Cancelar")
  return str(input("\nDigite o número da opção escolhida: "))

In [15]:
def randomRegister():
  choice = randomRegisterMenu()
  while choice != "4":
    match choice:
      case "1":
        processSingleRegister(all_transactions[all_transactions.Class==(randint(0,1))].sample())
        break
      case "2":
        processSingleRegister(frauds.sample())
        break
      case "3":
        processSingleRegister(valid_transactions.sample())
        break
      case _:
        print("\nOpção Inválida, digite novamente")
        choice = randomRegisterMenu()

#### Posição Relativa

In [16]:
def getClassMenu():
  choice = -1
  while choice < 0 or choice > 2:
    print("\nEscolha uma opção: ")
    print("\n1 - Transação Válida")
    print("2 - Transação Fraudulenta")
    print("3 - Cancelar")

    choice = int(input("\nDigite uma opção: ")) - 1

    if choice == 2:
      break
    elif choice == 0 or choice == 1:
      return choice
    else:
      print("\nOpção inválida, digite novamente")

In [17]:
def relativePositionMenu(register_class):
  position = -1
  list_size = len(all_transactions[all_transactions.Class==register_class])
  register_type = ""
  if register_class == 1:
    register_type = "FRAUDULENTA"
  else:
    register_type = "VÁLIDA"

  while position < 0 or position >= list_size:
    position = int(input(f'\nDigite a posição relativa da transação {register_type} (de 1 a {list_size}): ')) - 1
    if position < 0 or position >= list_size:
      print('\nPosição inválida, digite novamente')
    else:
      return position

In [18]:
def relativePosition():
  register_class = getClassMenu()
  if register_class == None:
    return
  relative_position = relativePositionMenu(register_class)
  register_index = all_transactions.index[all_transactions.Class==register_class].tolist()[relative_position]
  processSingleRegister(all_transactions[all_transactions.index==register_index])

#### Posição absoluta

In [19]:
def getAbsolutePosition():
  position = -1
  list_length = len(all_transactions)

  while position < 0 or position > list_length:
    position = int(input(f'\nDigite a posição desejada (de 1 a {list_length}): ')) - 1
    if position >= 0 and position < list_length:
      return position
    print("\nOpção inválida, digite novamente")

In [20]:
def absolutePosition():
  position = getAbsolutePosition()
  processSingleRegister(all_transactions[all_transactions.index==position])

#### Validação de Transação Única

In [21]:
def singleTransactionValidationMenu():
  print("\nEscolha uma opcão: \n")
  print("1 - Escolher aleatoriamente um registro")
  print("2 - Escolher a posição relativa do registro")
  print("3 - Escolher a posição exata do registro")
  print("4 - Cancelar")
  return str(input("\nDigite o número da opção escolhida: "))

In [22]:
def singleTransactionValidation():
  choice = singleTransactionValidationMenu()
  while choice != "4":
    match choice:
      case "1":
        randomRegister()
        break
      case "2":
        relativePosition()
        break
      case "3":
        absolutePosition()
        break
      case _:
        print("\nOpção Inválida, digite novamente")
        choice = singleTransactionValidationMenu()

#### Várias transações

In [72]:
def predictManyRegisters(model_location, registers):
  model = joblib.load(model_location)

  start_time = time.time()
  model_prediction = model.predict(registers.drop(columns=['Time', 'V5', 'V6', 'V7','V8','V13','V15', 'V20','V21','V22','V23', 'V24','V25','V26','V27','V28','Amount','Class']))
  execution_time = time.time() - start_time

  average_time = execution_time/len(registers)

  true_values = registers.Class.tolist()
  confusion_mtrx = confusion_matrix(true_values, model_prediction)

  print(f"\nQuantidade total de transações: {len(registers)} ({len(registers[registers.Class==1])} fraudes/ {len(registers[registers.Class==0])} válidas)")
  print(f'Verdadeiros negativos: {confusion_mtrx[0][0]} - Falsos negativos: {confusion_mtrx[1][0]}')
  print(f'Verdadeiros positivos: {confusion_mtrx[1][1]} - Falsos positivos: {confusion_mtrx[0][1]}')

  print("\nMétricas: ")
  print(f'Accuracy: {"{:.2f}".format(accuracy_score(true_values, model_prediction))}%')
  print(f'F1: {"{:.2f}".format(f1_score(true_values, model_prediction))}%')
  print(f'Precision: {"{:.2f}".format(precision_score(true_values, model_prediction))}%')
  print(f'Recall: {"{:.2f}".format(recall_score(true_values, model_prediction))}%')
  print(f'Roc Auc: {"{:.2f}".format(roc_auc_score(true_values, model_prediction))}%')

  print(f'\nTempo total de execução: {"{:.2f}".format(execution_time*1000)} Milisegundos')
  print(f'Tempo médio de execução: {"{:.5f}".format(average_time*1000)} Milisegundos')

In [36]:
def processManyRegisters(registers):
  model_location = chooseModel()
  if model_location == None:
    return
  predictManyRegisters(model_location, registers)

In [48]:
def getRandomTransactions(valid_qtt, fraud_qtt):
  valid_samples = valid_transactions.sample(valid_qtt)
  fraud_samples = frauds.sample(fraud_qtt)
  samples = pnd.concat([valid_samples, fraud_samples])
  return samples

In [53]:
def manyTransactionsMenu():
  valid_qtt = -1
  fraud_qtt = -1
  valid_length = len(valid_transactions)
  fraud_length = len(frauds)

  while valid_qtt < 1 or valid_qtt > valid_length:
    valid_qtt = int(input(f"\nDigite a quantidade total de transações válidas (de 1 a {valid_length}): "))
    if valid_qtt < 1 or valid_qtt > valid_length:
      print("\nQuantidade inválida, digite novamente")

  while fraud_qtt < 1 or fraud_qtt > fraud_length:
    fraud_qtt = int(input(f"\nDigite a quantidade total de transações fraudulentas (de 1 a {fraud_length}): "))
    if fraud_qtt < 1 or fraud_qtt > fraud_length:
      print("\nQuantidade inválida, digite novamente")

  return valid_qtt, fraud_qtt

In [43]:
def manyTransactions():
  valid_qtt, fraud_qtt = manyTransactionsMenu()
  transactions = getRandomTransactions(valid_qtt, fraud_qtt)
  processManyRegisters(transactions)

### Programa:

In [73]:
all_transactions = getTransactions()
frauds = all_transactions[all_transactions.Class==1]
valid_transactions = all_transactions[all_transactions.Class==0]

option = 0
while option != "3":
  option = printMenu()

  match option:
    case "1":
      singleTransactionValidation()
    case "2":
      manyTransactions()
    case "3":
      break
    case _:
      print("\nOpção inválida, digite novamente")


Qual operação que deseja realizar?: 

1 - Validar transação única
2 - Validar várias transações
3 - Sair

Digite o número da operação que deseja realizar: 2

Digite a quantidade total de transações válidas (de 1 a 284315): 500

Digite a quantidade total de transações fraudulentas (de 1 a 492): 492

Escolha uma opção de modelo:

0 - Cancelar
1 - Gaussian Naive Bayes 500 transações válidas
2 - Gaussian Naive Bayes 3000 transações válidas
3 - Gaussian Naive Bayes 1000 transações válidas
4 - K-Nearest Neighbors 500 transações válidas
5 - K-Nearest Neighbors 1000 transações válidas
6 - K-Nearest Neighbors 3000 transações válidas
7 - Support Vector Classifier 500 transações válidas
8 - Support Vector Classifier 1000 transações válidas
9 - Support Vector Classifier 3000 transações válidas

Digite o número do modelo desejado: 5

Quantidade total de transações: 992 (492 fraudes/ 500 válidas)
Verdadeiros negativos: 498 - Falsos negativos: 54
Verdadeiros positivos: 438 - Falsos positivos: 2

Mét